In [1]:
import uuid
import re
import sqlite3
from datetime import datetime, timedelta
import os
import csv
import json

GST_RATE = 0.18
DB_NAME = "car_mod.db"

# =========================
# MODIFICATION CLASSES
# =========================

class Modification:
    def __init__(self, name, price, category):
        self.name = name
        self.price = float(price)
        self.category = category

    def info(self):
        return f"{self.name:<30} ₹{self.price:>8.2f}"


class PerformanceMod(Modification):
    def __init__(self, name, price):
        super().__init__(name, price, "Performance")


class AestheticMod(Modification):
    def __init__(self, name, price):
        super().__init__(name, price, "Aesthetic")


class TechMod(Modification):
    def __init__(self, name, price):
        super().__init__(name, price, "Technology")


class SafetyMod(Modification):
    def __init__(self, name, price):
        super().__init__(name, price, "Safety")


class ComfortMod(Modification):
    def __init__(self, name, price):
        super().__init__(name, price, "Comfort")


class ColorMod(Modification):
    def __init__(self, name, price):
        super().__init__(name, price, "Color")


# =========================
# STUDIO MANAGER
# =========================

class ModificationStudio:
    def __init__(self):
        self.conn = sqlite3.connect(DB_NAME)
        self.cursor = self.conn.cursor()
        self.create_tables()
        self.load_mods()
        self.load_colors()
        self.load_customers()
        
        # Create necessary directories
        os.makedirs("bills", exist_ok=True)
        os.makedirs("exports", exist_ok=True)

    # -------------------------
    # DATABASE - Enhanced Structure
    # -------------------------

    def create_tables(self):
        # Customers table with more details
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS customers (
            customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
            email TEXT UNIQUE NOT NULL,
            name TEXT NOT NULL,
            phone TEXT,
            address TEXT,
            city TEXT,
            state TEXT,
            pincode TEXT,
            total_visits INTEGER DEFAULT 1,
            total_spent REAL DEFAULT 0,
            first_visit TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            last_visit TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            loyalty_points INTEGER DEFAULT 0,
            status TEXT DEFAULT 'active'
        )
        """)

        # Cars table (customer can have multiple cars)
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS cars (
            car_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_email TEXT NOT NULL,
            car_model TEXT NOT NULL,
            car_make TEXT,
            car_year INTEGER,
            car_vin TEXT,
            registration_date TEXT,
            car_color TEXT,
            FOREIGN KEY (customer_email) REFERENCES customers(email)
        )
        """)

        # Modifications master list
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS modifications (
            mod_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT UNIQUE NOT NULL,
            price REAL NOT NULL,
            category TEXT NOT NULL,
            description TEXT,
            stock_quantity INTEGER DEFAULT 100,
            is_active BOOLEAN DEFAULT 1,
            created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)

        # Bills table
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS bills (
            bill_id TEXT PRIMARY KEY,
            customer_email TEXT NOT NULL,
            car_id INTEGER,
            bill_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            bill_number INTEGER,
            subtotal REAL NOT NULL,
            discount REAL DEFAULT 0,
            discount_percent REAL DEFAULT 0,
            gst REAL NOT NULL,
            gst_rate REAL DEFAULT 0.18,
            total REAL NOT NULL,
            payment_method TEXT DEFAULT 'Cash',
            payment_status TEXT DEFAULT 'Paid',
            notes TEXT,
            FOREIGN KEY (customer_email) REFERENCES customers(email),
            FOREIGN KEY (car_id) REFERENCES cars(car_id)
        )
        """)

        # Bill items (modifications applied)
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS bill_items (
            item_id INTEGER PRIMARY KEY AUTOINCREMENT,
            bill_id TEXT NOT NULL,
            mod_id INTEGER,
            mod_name TEXT NOT NULL,
            mod_category TEXT NOT NULL,
            quantity INTEGER DEFAULT 1,
            price REAL NOT NULL,
            total_price REAL NOT NULL,
            FOREIGN KEY (bill_id) REFERENCES bills(bill_id),
            FOREIGN KEY (mod_id) REFERENCES modifications(mod_id)
        )
        """)

        # Exit logs table (for tracking when customers exit)
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS exit_logs (
            log_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_email TEXT NOT NULL,
            exit_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            exit_reason TEXT,
            session_duration_seconds INTEGER,
            modifications_selected INTEGER DEFAULT 0,
            bill_generated BOOLEAN DEFAULT 0,
            FOREIGN KEY (customer_email) REFERENCES customers(email)
        )
        """)

        # Appointments table
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS appointments (
            appointment_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_email TEXT NOT NULL,
            car_id INTEGER,
            appointment_date DATE NOT NULL,
            appointment_time TEXT NOT NULL,
            service_type TEXT,
            status TEXT DEFAULT 'Scheduled',
            notes TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (customer_email) REFERENCES customers(email),
            FOREIGN KEY (car_id) REFERENCES cars(car_id)
        )
        """)

        # Create indexes for better performance
        self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_customers_email ON customers(email)")
        self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_bills_date ON bills(bill_date)")
        self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_bills_customer ON bills(customer_email)")

        self.conn.commit()

    # -------------------------
    # LOAD DATA
    # -------------------------

    def load_mods(self):
        """Load modifications from database or create default"""
        self.cursor.execute("SELECT mod_id, name, price, category FROM modifications WHERE is_active = 1")
        db_mods = self.cursor.fetchall()
        
        if not db_mods:
            # Create default modifications if none exist
            default_mods = [
                ("Stage 2 ECU Remap", 1200, "Performance", "Engine performance upgrade"),
                ("Turbo Charger Kit", 3500, "Performance", "Adds turbo for more power"),
                ("Carbon Fiber Spoiler", 750, "Aesthetic", "Aerodynamic enhancement"),
                ("Ceramic Coating", 1200, "Aesthetic", "Premium paint protection"),
                ("Digital Instrument Cluster", 1400, "Technology", "Modern digital dashboard"),
                ("Heads-Up Display", 1100, "Technology", "Project information on windshield"),
                ("Advanced Brake System", 1600, "Safety", "High-performance brakes"),
                ("Blind Spot Monitor", 900, "Safety", "Detects vehicles in blind spot"),
                ("Premium Leather Seats", 2000, "Comfort", "Luxury seating"),
                ("Dual Zone Climate Control", 1300, "Comfort", "Individual temperature zones"),
                ("Premium Sound System", 1500, "Comfort", "High-end audio system"),
                ("LED Headlight Kit", 800, "Aesthetic", "Brighter LED headlights"),
                ("Performance Exhaust", 1800, "Performance", "Enhanced exhaust system"),
                ("Sports Suspension", 2200, "Performance", "Improved handling"),
                ("Parking Sensors", 600, "Safety", "Front and rear sensors"),
                ("Rear View Camera", 700, "Safety", "High-definition camera"),
                ("Sunroof Installation", 2500, "Comfort", "Power sunroof"),
                ("Window Tinting", 500, "Aesthetic", "UV protection film")
            ]
            
            for name, price, category, description in default_mods:
                self.cursor.execute("""
                    INSERT OR IGNORE INTO modifications (name, price, category, description) 
                    VALUES (?, ?, ?, ?)
                """, (name, price, category, description))
            self.conn.commit()
            
            # Reload from database
            self.cursor.execute("SELECT mod_id, name, price, category FROM modifications WHERE is_active = 1")
            db_mods = self.cursor.fetchall()
        
        # Create modification objects
        self.mods = []
        category_map = {
            "Performance": PerformanceMod,
            "Aesthetic": AestheticMod,
            "Technology": TechMod,
            "Safety": SafetyMod,
            "Comfort": ComfortMod,
            "Color": ColorMod
        }
        
        for mod_id, name, price, category in db_mods:
            if category in category_map:
                self.mods.append(category_map[category](name, price))
                self.mods[-1].mod_id = mod_id

    def load_colors(self):
        """Load colors from database or create default"""
        self.cursor.execute("SELECT mod_id, name, price FROM modifications WHERE category = 'Color' AND is_active = 1")
        db_colors = self.cursor.fetchall()
        
        if not db_colors:
            # Create default colors
            default_colors = [
                ("Midnight Purple", 2500, "Metallic purple with sparkles"),
                ("Gunmetal Grey", 2300, "Matte grey finish"),
                ("Candy Apple Red", 2200, "Deep red metallic"),
                ("Satin Gold", 2800, "Premium gold finish"),
                ("Gloss Black", 2000, "Deep black paint"),
                ("Pearl White", 2400, "Pearlescent white"),
                ("Electric Blue", 2600, "Vibrant blue metallic"),
                ("Racing Green", 2100, "British racing green")
            ]
            
            for name, price, description in default_colors:
                self.cursor.execute("""
                    INSERT OR IGNORE INTO modifications (name, price, category, description) 
                    VALUES (?, ?, 'Color', ?)
                """, (name, price, description))
            self.conn.commit()
            
            # Reload from database
            self.cursor.execute("SELECT mod_id, name, price FROM modifications WHERE category = 'Color' AND is_active = 1")
            db_colors = self.cursor.fetchall()
        
        # Create color objects
        self.colors = [ColorMod(name, price) for mod_id, name, price in db_colors]
        for i, (mod_id, name, price) in enumerate(db_colors):
            self.colors[i].mod_id = mod_id

    def load_customers(self):
        """Load customers from database"""
        self.cursor.execute("""
            SELECT email, name, phone, total_visits, total_spent, loyalty_points 
            FROM customers
        """)
        self.customers = {row[0]: {
            'name': row[1],
            'phone': row[2],
            'visits': row[3],
            'total_spent': row[4],
            'loyalty_points': row[5]
        } for row in self.cursor.fetchall()}

    # -------------------------
    # UTILITY FUNCTIONS
    # -------------------------

    def safe_text(self, text):
        return re.sub(r"[^A-Za-z0-9]", "_", text)

    def valid_name(self, name):
        return bool(re.fullmatch(r"[A-Za-z ]{2,50}", name))

    def valid_email(self, email):
        return bool(re.fullmatch(r"[\w.-]+@[\w.-]+\.\w{2,}", email))

    def valid_phone(self, phone):
        return bool(re.fullmatch(r"\+?[0-9\s\-]{10,15}", phone))

    def valid_pincode(self, pincode):
        return bool(re.fullmatch(r"[0-9]{6}", pincode))

    def get_int_input(self, prompt, min_val=0, max_val=None):
        """Safe integer input with validation"""
        while True:
            try:
                value = int(input(prompt))
                if (min_val is not None and value < min_val) or \
                   (max_val is not None and value > max_val):
                    print(f"Please enter a number between {min_val} and {max_val}")
                    continue
                return value
            except ValueError:
                print("Please enter a valid number")

    def get_next_bill_number(self):
        """Generate next bill number"""
        self.cursor.execute("SELECT MAX(bill_number) FROM bills")
        result = self.cursor.fetchone()[0]
        return (result or 0) + 1

    # -------------------------
    # CUSTOMER MANAGEMENT
    # -------------------------

    def get_customer_info(self):
        """Get or create customer information"""
        while True:
            email = input("Enter email: ").strip().lower()
            if not self.valid_email(email):
                print("Invalid email format. Please try again.")
                continue
            
            if email in self.customers:
                customer = self.customers[email]
                print(f"\n{'='*60}")
                print(f"Welcome back {customer['name']}!")
                print(f"Your loyalty points: {customer['loyalty_points']}")
                print(f"Total visits: {customer['visits']}")
                print(f"Total spent: ₹{customer['total_spent']:.2f}")
                print(f"{'='*60}\n")
                return email, customer['name'], customer['phone'], True
            
            # New customer
            print("\n=== NEW CUSTOMER REGISTRATION ===")
            while True:
                name = input("Enter your full name: ").strip()
                if self.valid_name(name):
                    break
                print("Invalid name. Use only letters and spaces (2-50 characters).")
            
            phone = ""
            while True:
                phone = input("Enter phone number: ").strip()
                if not phone or self.valid_phone(phone):
                    break
                print("Invalid phone number format.")
            
            address = input("Enter address (optional): ").strip()
            city = input("Enter city (optional): ").strip()
            state = input("Enter state (optional): ").strip()
            
            pincode = ""
            while True:
                pincode = input("Enter pincode (optional): ").strip()
                if not pincode or self.valid_pincode(pincode):
                    break
                print("Invalid pincode format. Should be 6 digits.")
            
            # Insert new customer
            self.cursor.execute("""
                INSERT INTO customers (email, name, phone, address, city, state, pincode) 
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (email, name, phone, address, city, state, pincode))
            self.conn.commit()
            
            self.customers[email] = {
                'name': name,
                'phone': phone,
                'visits': 1,
                'total_spent': 0,
                'loyalty_points': 0
            }
            
            print(f"\n✓ Customer registered successfully!")
            return email, name, phone, False

    def update_customer_visit(self, email, amount_spent=0):
        """Update customer visit count and spending"""
        points_earned = int(amount_spent / 100)
        
        self.cursor.execute("""
            UPDATE customers 
            SET total_visits = total_visits + 1,
                total_spent = total_spent + ?,
                last_visit = CURRENT_TIMESTAMP,
                loyalty_points = loyalty_points + ?
            WHERE email = ?
        """, (amount_spent, points_earned, email))
        self.conn.commit()
        
        # Update local cache
        if email in self.customers:
            self.customers[email]['visits'] += 1
            self.customers[email]['total_spent'] += amount_spent
            self.customers[email]['loyalty_points'] += points_earned
        
        return points_earned

    # -------------------------
    # CAR MANAGEMENT
    # -------------------------

    def get_or_create_car(self, customer_email):
        """Get existing car or create new one"""
        # Check if customer has cars
        self.cursor.execute("""
            SELECT car_id, car_model, car_make, car_year FROM cars 
            WHERE customer_email = ?
        """, (customer_email,))
        cars = self.cursor.fetchall()
        
        if cars:
            print("\nYour existing cars:")
            for i, (car_id, model, make, year) in enumerate(cars, 1):
                make_display = f" {make}" if make else ""
                year_display = f" ({year})" if year else ""
                print(f"{i}. {model}{make_display}{year_display}")
            print(f"{len(cars) + 1}. Add new car")
            print(f"{len(cars) + 2}. Skip car selection")
            
            choice = self.get_int_input("Select option: ", 1, len(cars) + 2)
            
            if choice <= len(cars):
                return cars[choice - 1][0], cars[choice - 1][1]
            elif choice == len(cars) + 1:
                # Add new car
                return self.add_new_car(customer_email)
            else:
                # Skip car selection
                return None, input("Enter car model (for billing): ").strip()
        
        # Add new car
        return self.add_new_car(customer_email)

    def add_new_car(self, customer_email):
        """Add a new car for customer"""
        print("\n=== ADD NEW CAR ===")
        car_model = input("Enter car model: ").strip()
        car_make = input("Enter car make (optional): ").strip()
        
        car_year = ""
        while True:
            car_year = input("Enter car year (optional): ").strip()
            if not car_year or (car_year.isdigit() and 1900 <= int(car_year) <= datetime.now().year + 1):
                break
            print(f"Invalid year. Must be between 1900 and {datetime.now().year + 1}")
        
        car_year = int(car_year) if car_year.isdigit() else None
        car_color = input("Enter car color (optional): ").strip()
        
        self.cursor.execute("""
            INSERT INTO cars (customer_email, car_model, car_make, car_year, car_color) 
            VALUES (?, ?, ?, ?, ?)
        """, (customer_email, car_model, car_make, car_year, car_color))
        self.conn.commit()
        
        return self.cursor.lastrowid, car_model

    # -------------------------
    # MAIN MENU
    # -------------------------

    def menu(self):
        session_start = datetime.now()
        session_email = None
        
        while True:
            print("\n" + "="*60)
            print("🚗 CUSTOM CAR MODIFICATION STUDIO 🚗")
            print("="*60)
            print("1. New Modification Build")
            print("2. View Available Modifications & Pricing")
            print("3. View Customer Profile & History")
            print("4. Schedule Appointment")
            print("5. View Appointments")
            print("6. Generate Reports")
            print("7. Export Data")
            print("8. Exit Program")
            
            choice = input("\nChoose option: ").strip()
            
            if choice == "1":
                session_email = self.start_build()
            elif choice == "2":
                self.show_pricing_summary()
            elif choice == "3":
                self.view_customer_history_menu()
            elif choice == "4":
                self.schedule_appointment()
            elif choice == "5":
                self.view_appointments()
            elif choice == "6":
                self.generate_reports_menu()
            elif choice == "7":
                self.export_data_menu()
            elif choice == "8":
                # Log exit
                exit_reason = input("Reason for exit (optional): ").strip()
                session_duration = (datetime.now() - session_start).seconds
                
                # Store exit data in database
                self.cursor.execute("""
                    INSERT INTO exit_logs (customer_email, exit_reason, session_duration_seconds)
                    VALUES (?, ?, ?)
                """, (session_email or "system_exit", exit_reason, session_duration))
                self.conn.commit()
                
                print("\n" + "="*60)
                print("Thank you for using Custom Car Modification Studio!")
                print("All data has been saved to the database.")
                print("="*60)
                break
            else:
                print("Invalid option, please try again")

    # -------------------------
    # BUILD PROCESS
    # -------------------------

    def start_build(self):
        session_start = datetime.now()
        session_email = None
        
        try:
            # Get customer info
            email, name, phone, is_returning = self.get_customer_info()
            session_email = email
            
            # Get car info
            car_id, car_model = self.get_or_create_car(email)
            
            selections = []
            selected_mod_ids = []
            print("\n=== SELECT MODIFICATIONS ===")
            print("Enter 0 to finish, 00 to cancel")
            
            while True:
                print("\nAvailable Modifications:")
                for i, m in enumerate(self.mods, 1):
                    category_abbr = m.category[:3].upper()
                    print(f"{i:2}. [{category_abbr}] {m.name:<35} ₹{m.price:>8.2f}")
                print("\n 0. Finish Selection")
                print("00. Cancel and Exit")
                
                ch_input = input("\nSelect modification (or '0' to finish): ").strip()
                
                if ch_input == "0":
                    break
                elif ch_input == "00":
                    # Log exit without purchase
                    session_duration = (datetime.now() - session_start).seconds
                    self.log_customer_exit(email, session_duration, len(selections), "cancelled_build")
                    print("\nBuild cancelled. Data saved.")
                    return email
                elif ch_input.isdigit():
                    ch = int(ch_input)
                    if 1 <= ch <= len(self.mods):
                        mod = self.mods[ch - 1]
                        if mod in selections:
                            print("Already selected")
                            continue
                        selections.append(mod)
                        selected_mod_ids.append(mod.mod_id)
                        print(f"✓ Added: {mod.name}")
                    else:
                        print("Invalid selection")
                else:
                    print("Invalid input")
            
            color = None
            color_mod_id = None
            if selections or True:  # Allow color selection even if no other mods
                print("\n=== SELECT COLOR (optional) ===")
                for i, c in enumerate(self.colors, 1):
                    print(f"{i:2}. {c.name:<35} ₹{c.price:>8.2f}")
                print(" 0. Skip")
                print("00. Cancel and Exit")
                
                ch_input = input("Select color: ").strip()
                
                if ch_input == "00":
                    # Log exit without purchase
                    session_duration = (datetime.now() - session_start).seconds
                    self.log_customer_exit(email, session_duration, len(selections), "cancelled_build")
                    print("Build cancelled. Data saved.")
                    return email
                elif ch_input.isdigit() and ch_input != "0":
                    ch = int(ch_input)
                    if 1 <= ch <= len(self.colors):
                        color = self.colors[ch - 1]
                        color_mod_id = color.mod_id
            
            if not selections and not color:
                print("\nNo modifications selected. Build cancelled.")
                # Log exit without purchase
                session_duration = (datetime.now() - session_start).seconds
                self.log_customer_exit(email, session_duration, 0, "no_selection")
                return email
            
            # Generate bill
            self.generate_bill(email, name, car_id, car_model, selections, selected_mod_ids, color, color_mod_id)
            
            return email
            
        except KeyboardInterrupt:
            print("\n\nBuild interrupted by user")
            if session_email:
                session_duration = (datetime.now() - session_start).seconds
                self.log_customer_exit(session_email, session_duration, len(selections) if 'selections' in locals() else 0, "keyboard_interrupt")
            return session_email

    def log_customer_exit(self, email, session_duration, mods_selected, reason="user_exit"):
        """Log when a customer exits without completing purchase"""
        self.cursor.execute("""
            INSERT INTO exit_logs (customer_email, exit_reason, session_duration_seconds, modifications_selected)
            VALUES (?, ?, ?, ?)
        """, (email, reason, session_duration, mods_selected))
        self.conn.commit()

    # -------------------------
    # BILL GENERATION
    # -------------------------

    def generate_bill(self, email, name, car_id, car_model, mods, mod_ids, color, color_mod_id):
        """Generate and save bill with all details"""
        bill_date = datetime.now()
        timestamp = bill_date.strftime("%d-%m-%Y %H:%M:%S")
        subtotal = 0
        
        # Calculate subtotal
        for m in mods:
            subtotal += m.price
        
        if color:
            subtotal += color.price
        
        # Apply discounts
        discount_amount = 0
        discount_percent = 0
        
        if len(mods) >= 3:
            discount_percent += 10
        if self.customers.get(email, {}).get('visits', 0) > 1:
            discount_percent += 5
        
        if discount_percent > 0:
            discount_amount = subtotal * (discount_percent / 100)
            subtotal -= discount_amount
        
        # Calculate GST
        gst = subtotal * GST_RATE
        total = subtotal + gst
        
        # Generate bill ID and number
        bill_id = f"BILL-{datetime.now().strftime('%Y%m%d')}-{str(uuid.uuid4())[:6].upper()}"
        bill_number = self.get_next_bill_number()
        
        # Get payment method
        print("\n=== PAYMENT DETAILS ===")
        payment_methods = ["Cash", "Credit Card", "Debit Card", "UPI", "Net Banking"]
        for i, method in enumerate(payment_methods, 1):
            print(f"{i}. {method}")
        
        payment_choice = self.get_int_input("Select payment method: ", 1, len(payment_methods))
        payment_method = payment_methods[payment_choice - 1]
        
        notes = input("Any special notes (optional): ").strip()
        
        # Save bill to database
        self.cursor.execute("""
            INSERT INTO bills (bill_id, customer_email, car_id, bill_date, bill_number, 
                             subtotal, discount, discount_percent, gst, gst_rate, total, 
                             payment_method, notes)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (bill_id, email, car_id, timestamp, bill_number, 
              subtotal + discount_amount, discount_amount, discount_percent, 
              gst, GST_RATE, total, payment_method, notes))
        
        # Save bill items
        for i, m in enumerate(mods):
            self.cursor.execute("""
                INSERT INTO bill_items (bill_id, mod_id, mod_name, mod_category, price, total_price)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (bill_id, mod_ids[i], m.name, m.category, m.price, m.price))
        
        if color:
            self.cursor.execute("""
                INSERT INTO bill_items (bill_id, mod_id, mod_name, mod_category, price, total_price)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (bill_id, color_mod_id, color.name, color.category, color.price, color.price))
        
        # Update customer visit and spending
        points_earned = self.update_customer_visit(email, total)
        
        self.conn.commit()
        
        # Generate bill file
        bill_filename = self.generate_bill_file(
            bill_id, bill_number, email, name, car_model, mods, color, 
            subtotal, discount_amount, discount_percent, gst, total, 
            timestamp, payment_method, points_earned
        )
        
        # Display bill summary
        self.display_bill_summary(
            bill_id, bill_number, name, car_model, total, 
            discount_percent, points_earned, bill_filename
        )
        
        # Log successful bill generation
        session_duration = (datetime.now() - datetime.strptime(timestamp, "%d-%m-%Y %H:%M:%S")).seconds
        self.cursor.execute("""
            INSERT INTO exit_logs (customer_email, exit_reason, session_duration_seconds, 
                                 modifications_selected, bill_generated)
            VALUES (?, ?, ?, ?, ?)
        """, (email, "bill_generated", session_duration, len(mods) + (1 if color else 0), 1))
        self.conn.commit()

    def generate_bill_file(self, bill_id, bill_number, email, name, car_model, mods, color, 
                          subtotal, discount, discount_percent, gst, total, 
                          timestamp, payment_method, points_earned):
        """Generate a text file bill"""
        safe_name = self.safe_text(name)
        date_str = datetime.now().strftime("%Y%m%d")
        filename = f"bills/{date_str}_{bill_number}_{safe_name}.txt"
        
        bill_lines = [
            "="*70,
            " " * 20 + "CUSTOM CAR MODIFICATION STUDIO",
            "="*70,
            f"{'Bill No:':<15} {bill_number}",
            f"{'Bill ID:':<15} {bill_id}",
            f"{'Date:':<15} {timestamp}",
            "="*70,
            f"{'Customer:':<15} {name}",
            f"{'Email:':<15} {email}",
            f"{'Car Model:':<15} {car_model}",
            f"{'Payment:':<15} {payment_method}",
            "="*70,
            " " * 5 + "MODIFICATIONS",
            "-"*70,
            f"{'No.':<5} {'Description':<40} {'Price':>10} {'Total':>10}"
        ]
        
        item_no = 1
        for m in mods:
            bill_lines.append(f"{item_no:<5} {m.name:<40} ₹{m.price:>8.2f} ₹{m.price:>8.2f}")
            item_no += 1
        
        if color:
            bill_lines.append(f"{item_no:<5} Paint - {color.name:<34} ₹{color.price:>8.2f} ₹{color.price:>8.2f}")
        
        bill_lines.extend([
            "-"*70,
            f"{'Subtotal:':<55} ₹{subtotal + discount:>10.2f}",
            f"{f'Discount ({discount_percent}%):':<55} -₹{discount:>10.2f}"
        ])
        
        if points_earned > 0:
            bill_lines.append(f"{'Loyalty Points Earned:':<55} {points_earned:>10}")
        
        bill_lines.extend([
            f"{'GST (18%):':<55} ₹{gst:>10.2f}",
            "="*70,
            f"{'TOTAL AMOUNT:':<55} ₹{total:>10.2f}",
            "="*70,
            "\n" + " " * 10 + "Thank you for your business!",
            " " * 5 + "Visit again for more modifications!"
        ])
        
        with open(filename, "w", encoding="utf-8") as f:
            f.write("\n".join(bill_lines))
        
        return filename

    def display_bill_summary(self, bill_id, bill_number, name, car_model, total, 
                            discount_percent, points_earned, bill_filename):
        """Display bill summary to user"""
        print("\n" + "="*70)
        print(" " * 25 + "BILL GENERATED")
        print("="*70)
        print(f"✓ Bill Number: {bill_number}")
        print(f"✓ Bill ID: {bill_id}")
        print(f"✓ Customer: {name}")
        print(f"✓ Car: {car_model}")
        print(f"✓ Total Amount: ₹{total:.2f}")
        if discount_percent > 0:
            print(f"✓ Discount Applied: {discount_percent}%")
        if points_earned > 0:
            print(f"✓ Loyalty Points Earned: {points_earned}")
        print(f"✓ Bill Saved As: {bill_filename}")
        print("="*70)

    # -------------------------
    # VIEW FUNCTIONS
    # -------------------------

    def show_pricing_summary(self):
        """Display all available modifications with categories"""
        print("\n" + "="*80)
        print(" " * 25 + "AVAILABLE MODIFICATIONS & PRICING")
        print("="*80)
        
        # Group by category
        categories = {}
        for mod in self.mods + self.colors:
            if mod.category not in categories:
                categories[mod.category] = []
            categories[mod.category].append(mod)
        
        for category, mods in sorted(categories.items()):
            print(f"\n{category.upper()} ({len(mods)} items):")
            print("-"*80)
            for mod in sorted(mods, key=lambda x: x.price, reverse=True):
                print(f"  • {mod.name:<45} ₹{mod.price:>8.2f}")
        
        print("\n" + "="*80)
        print("DISCOUNTS & OFFERS:")
        print("-"*80)
        print("  • 10% discount for 3 or more modifications")
        print("  • 5% loyalty discount for returning customers")
        print("  • Combined discounts up to 15%")
        print("  • Earn 1 loyalty point for every ₹100 spent")
        print("="*80)

    def view_customer_history_menu(self):
        """Menu for viewing customer history"""
        email = input("Enter customer email (or leave blank for all): ").strip().lower()
        
        if email:
            self.view_customer_history(email)
        else:
            self.view_all_customers()

    def view_customer_history(self, email):
        """View customer modification history"""
        # Get customer basic info
        self.cursor.execute("""
            SELECT c.name, c.phone, c.address, c.city, c.state, c.pincode,
                   c.total_visits, c.total_spent, c.loyalty_points,
                   c.first_visit, c.last_visit
            FROM customers c
            WHERE c.email = ?
        """, (email,))
        
        customer_info = self.cursor.fetchone()
        
        if not customer_info:
            print("Customer not found.")
            return
        
        print("\n" + "="*80)
        print(" " * 25 + "CUSTOMER PROFILE")
        print("="*80)
        print(f"{'Name:':<15} {customer_info[0]}")
        print(f"{'Email:':<15} {email}")
        print(f"{'Phone:':<15} {customer_info[1] or 'N/A'}")
        print(f"{'Address:':<15} {customer_info[2] or 'N/A'}, {customer_info[3] or ''}, {customer_info[4] or ''} - {customer_info[5] or ''}")
        print("-"*80)
        print(f"{'Total Visits:':<20} {customer_info[6]}")
        print(f"{'Total Spent:':<20} ₹{customer_info[7]:.2f}")
        print(f"{'Loyalty Points:':<20} {customer_info[8]}")
        print(f"{'First Visit:':<20} {customer_info[9]}")
        print(f"{'Last Visit:':<20} {customer_info[10]}")
        print("="*80)
        
        # Get customer's cars
        self.cursor.execute("""
            SELECT car_model, car_make, car_year, car_color
            FROM cars
            WHERE customer_email = ?
            ORDER BY car_year DESC
        """, (email,))
        
        cars = self.cursor.fetchall()
        if cars:
            print("\nCUSTOMER'S CARS:")
            print("-"*80)
            for i, (model, make, year, color) in enumerate(cars, 1):
                details = f"{model}"
                if make:
                    details += f" ({make})"
                if year:
                    details += f", {year}"
                if color:
                    details += f", {color}"
                print(f"{i}. {details}")
        
        # Get recent bills (last 10)
        self.cursor.execute("""
            SELECT b.bill_id, b.bill_date, b.total, b.payment_method,
                   ca.car_model,
                   GROUP_CONCAT(DISTINCT bi.mod_name, ', ') as modifications
            FROM bills b
            LEFT JOIN cars ca ON b.car_id = ca.car_id
            LEFT JOIN bill_items bi ON b.bill_id = bi.bill_id
            WHERE b.customer_email = ?
            GROUP BY b.bill_id
            ORDER BY b.bill_date DESC
            LIMIT 10
        """, (email,))
        
        recent_bills = self.cursor.fetchall()
        
        if recent_bills:
            print("\nRECENT BILLS (Last 10):")
            print("-"*80)
            for bill_id, bill_date, total, payment_method, car_model, mods in recent_bills:
                print(f"\n  Bill ID: {bill_id}")
                print(f"  Date: {bill_date}")
                print(f"  Car: {car_model}")
                print(f"  Total: ₹{total:.2f}")
                print(f"  Payment: {payment_method}")
                print(f"  Modifications: {mods[:100]}..." if len(mods) > 100 else f"  Modifications: {mods}")
        
        print("="*80)

    def view_all_customers(self):
        """View all customers summary"""
        self.cursor.execute("""
            SELECT email, name, total_visits, total_spent, loyalty_points, last_visit
            FROM customers
            ORDER BY total_spent DESC
            LIMIT 50
        """)
        
        customers = self.cursor.fetchall()
        
        if not customers:
            print("No customers found.")
            return
        
        print("\n" + "="*100)
        print(" " * 35 + "ALL CUSTOMERS SUMMARY")
        print("="*100)
        print(f"{'No.':<4} {'Name':<25} {'Email':<30} {'Visits':<8} {'Spent':<12} {'Points':<8} {'Last Visit':<12}")
        print("-"*100)
        
        total_customers = 0
        total_revenue = 0
        
        for i, (email, name, visits, spent, points, last_visit) in enumerate(customers, 1):
            print(f"{i:<4} {name[:24]:<25} {email[:29]:<30} {visits:<8} ₹{spent:<10.2f} {points:<8} {last_visit[:10]:<12}")
            total_customers += 1
            total_revenue += spent
        
        print("-"*100)
        print(f"Total Customers: {total_customers}")
        print(f"Total Revenue: ₹{total_revenue:.2f}")
        print(f"Average per Customer: ₹{total_revenue/total_customers if total_customers > 0 else 0:.2f}")
        print("="*100)

    # -------------------------
    # APPOINTMENT SYSTEM
    # -------------------------

    def schedule_appointment(self):
        """Schedule a new appointment"""
        print("\n=== SCHEDULE NEW APPOINTMENT ===")
        
        # Get customer email
        email = input("Enter customer email: ").strip().lower()
        
        # Check if customer exists
        self.cursor.execute("SELECT name FROM customers WHERE email = ?", (email,))
        customer = self.cursor.fetchone()
        
        if not customer:
            print("Customer not found. Please register first.")
            return
        
        # Get date
        while True:
            date_str = input("Enter appointment date (DD-MM-YYYY): ").strip()
            try:
                app_date = datetime.strptime(date_str, "%d-%m-%Y").date()
                if app_date < datetime.now().date():
                    print("Appointment date cannot be in the past.")
                    continue
                break
            except ValueError:
                print("Invalid date format. Use DD-MM-YYYY.")
        
        # Get time
        while True:
            time_str = input("Enter appointment time (HH:MM, 09:00 to 18:00): ").strip()
            try:
                datetime.strptime(time_str, "%H:%M")
                hour = int(time_str.split(":")[0])
                if hour < 9 or hour > 18:
                    print("Appointments only available from 09:00 to 18:00.")
                    continue
                break
            except ValueError:
                print("Invalid time format. Use HH:MM.")
        
        # Get car
        car_id, car_model = self.get_or_create_car(email)
        
        # Get service type
        service_types = ["Modification", "Consultation", "Maintenance", "Repair", "Inspection"]
        print("\nService Types:")
        for i, service in enumerate(service_types, 1):
            print(f"{i}. {service}")
        
        service_choice = self.get_int_input("Select service type: ", 1, len(service_types))
        service_type = service_types[service_choice - 1]
        
        notes = input("Any special notes (optional): ").strip()
        
        # Save appointment
        self.cursor.execute("""
            INSERT INTO appointments (customer_email, car_id, appointment_date, 
                                    appointment_time, service_type, notes)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (email, car_id, app_date.strftime("%Y-%m-%d"), time_str, service_type, notes))
        self.conn.commit()
        
        print(f"\n✓ Appointment scheduled for {customer[0]} on {date_str} at {time_str}")
        print(f"  Service: {service_type}")
        print(f"  Car: {car_model}")

    def view_appointments(self):
        """View upcoming appointments"""
        print("\n=== APPOINTMENTS ===")
        print("1. View Today's Appointments")
        print("2. View This Week's Appointments")
        print("3. View All Upcoming Appointments")
        print("4. Search by Date")
        
        choice = self.get_int_input("Select option: ", 1, 4)
        
        today = datetime.now().date()
        
        if choice == 1:
            # Today's appointments
            self.cursor.execute("""
                SELECT a.appointment_date, a.appointment_time, c.name, ca.car_model,
                       a.service_type, a.status
                FROM appointments a
                JOIN customers c ON a.customer_email = c.email
                LEFT JOIN cars ca ON a.car_id = ca.car_id
                WHERE a.appointment_date = ? AND a.status != 'Cancelled'
                ORDER BY a.appointment_time
            """, (today.strftime("%Y-%m-%d"),))
            appointments = self.cursor.fetchall()
            title = "TODAY'S APPOINTMENTS"
            
        elif choice == 2:
            # This week's appointments
            week_end = today + timedelta(days=7)
            self.cursor.execute("""
                SELECT a.appointment_date, a.appointment_time, c.name, ca.car_model,
                       a.service_type, a.status
                FROM appointments a
                JOIN customers c ON a.customer_email = c.email
                LEFT JOIN cars ca ON a.car_id = ca.car_id
                WHERE a.appointment_date BETWEEN ? AND ? 
                      AND a.status != 'Cancelled'
                ORDER BY a.appointment_date, a.appointment_time
            """, (today.strftime("%Y-%m-%d"), week_end.strftime("%Y-%m-%d")))
            appointments = self.cursor.fetchall()
            title = "THIS WEEK'S APPOINTMENTS"
            
        elif choice == 3:
            # All upcoming appointments
            self.cursor.execute("""
                SELECT a.appointment_date, a.appointment_time, c.name, ca.car_model,
                       a.service_type, a.status
                FROM appointments a
                JOIN customers c ON a.customer_email = c.email
                LEFT JOIN cars ca ON a.car_id = ca.car_id
                WHERE a.appointment_date >= ? AND a.status != 'Cancelled'
                ORDER BY a.appointment_date, a.appointment_time
            """, (today.strftime("%Y-%m-%d"),))
            appointments = self.cursor.fetchall()
            title = "ALL UPCOMING APPOINTMENTS"
            
        else:
            # Search by date
            date_str = input("Enter date to search (DD-MM-YYYY): ").strip()
            try:
                search_date = datetime.strptime(date_str, "%d-%m-%Y").date()
                self.cursor.execute("""
                    SELECT a.appointment_date, a.appointment_time, c.name, ca.car_model,
                           a.service_type, a.status
                    FROM appointments a
                    JOIN customers c ON a.customer_email = c.email
                    LEFT JOIN cars ca ON a.car_id = ca.car_id
                    WHERE a.appointment_date = ? AND a.status != 'Cancelled'
                    ORDER BY a.appointment_time
                """, (search_date.strftime("%Y-%m-%d"),))
                appointments = self.cursor.fetchall()
                title = f"APPOINTMENTS ON {date_str}"
            except ValueError:
                print("Invalid date format.")
                return
        
        if not appointments:
            print(f"\nNo appointments found.")
            return
        
        print(f"\n{title}")
        print("="*80)
        print(f"{'Date':<12} {'Time':<8} {'Customer':<20} {'Car':<15} {'Service':<15} {'Status':<10}")
        print("-"*80)
        
        for app_date, app_time, name, car_model, service_type, status in appointments:
            date_display = datetime.strptime(str(app_date), "%Y-%m-%d").strftime("%d-%m-%Y")
            print(f"{date_display:<12} {app_time:<8} {name[:19]:<20} {str(car_model)[:14]:<15} {service_type[:14]:<15} {status:<10}")
        
        print("="*80)
        print(f"Total Appointments: {len(appointments)}")

    # -------------------------
    # REPORTS
    # -------------------------

    def generate_reports_menu(self):
        """Menu for generating reports"""
        while True:
            print("\n=== REPORTS MENU ===")
            print("1. Sales Report")
            print("2. Customer Report")
            print("3. Modifications Report")
            print("4. Daily Summary")
            print("5. Monthly Report")
            print("6. Back to Main Menu")
            
            choice = self.get_int_input("Select option: ", 1, 6)
            
            if choice == 1:
                self.generate_sales_report()
            elif choice == 2:
                self.generate_customer_report()
            elif choice == 3:
                self.generate_modifications_report()
            elif choice == 4:
                self.generate_daily_summary()
            elif choice == 5:
                self.generate_monthly_report()
            elif choice == 6:
                break

    def generate_sales_report(self):
        """Generate sales report"""
        print("\n=== SALES REPORT ===")
        print("1. Today's Sales")
        print("2. This Week's Sales")
        print("3. This Month's Sales")
        print("4. Custom Date Range")
        
        choice = self.get_int_input("Select option: ", 1, 4)
        
        today = datetime.now().date()
        
        if choice == 1:
            date_condition = "DATE(bill_date) = DATE('now')"
            title = "TODAY'S SALES REPORT"
        elif choice == 2:
            date_condition = "DATE(bill_date) >= DATE('now', 'weekday 0', '-7 days')"
            title = "THIS WEEK'S SALES REPORT"
        elif choice == 3:
            date_condition = "strftime('%Y-%m', bill_date) = strftime('%Y-%m', 'now')"
            title = "THIS MONTH'S SALES REPORT"
        else:
            start_date = input("Enter start date (DD-MM-YYYY): ").strip()
            end_date = input("Enter end date (DD-MM-YYYY): ").strip()
            try:
                datetime.strptime(start_date, "%d-%m-%Y")
                datetime.strptime(end_date, "%d-%m-%Y")
                date_condition = f"DATE(bill_date) BETWEEN '{datetime.strptime(start_date, '%d-%m-%Y').strftime('%Y-%m-%d')}' AND '{datetime.strptime(end_date, '%d-%m-%Y').strftime('%Y-%m-%d')}'"
                title = f"SALES REPORT FROM {start_date} TO {end_date}"
            except ValueError:
                print("Invalid date format.")
                return
        
        # Generate report
        self.cursor.execute(f"""
            SELECT 
                DATE(b.bill_date) as bill_date,
                COUNT(*) as total_bills,
                SUM(b.total) as total_revenue,
                AVG(b.total) as avg_bill,
                SUM(b.discount) as total_discount,
                SUM(b.gst) as total_gst
            FROM bills b
            WHERE {date_condition}
            GROUP BY DATE(b.bill_date)
            ORDER BY bill_date DESC
        """)
        
        sales_data = self.cursor.fetchall()
        
        if not sales_data:
            print("No sales data found for the selected period.")
            return
        
        print(f"\n{title}")
        print("="*100)
        print(f"{'Date':<12} {'Bills':<8} {'Revenue':<15} {'Avg Bill':<12} {'Discount':<12} {'GST':<12}")
        print("-"*100)
        
        total_bills = 0
        total_revenue = 0
        total_discount = 0
        total_gst = 0
        
        for date_str, bills, revenue, avg_bill, discount, gst in sales_data:
            date_display = datetime.strptime(str(date_str), "%Y-%m-%d").strftime("%d-%m-%Y")
            print(f"{date_display:<12} {bills:<8} ₹{revenue:<13.2f} ₹{avg_bill:<10.2f} ₹{discount:<10.2f} ₹{gst:<10.2f}")
            total_bills += bills
            total_revenue += revenue
            total_discount += discount
            total_gst += gst
        
        print("-"*100)
        print(f"{'TOTAL':<12} {total_bills:<8} ₹{total_revenue:<13.2f} ₹{total_revenue/total_bills if total_bills > 0 else 0:<10.2f} ₹{total_discount:<10.2f} ₹{total_gst:<10.2f}")
        print("="*100)

    def generate_customer_report(self):
        """Generate customer report"""
        print("\n=== CUSTOMER REPORT ===")
        
        self.cursor.execute("""
            SELECT 
                COUNT(*) as total_customers,
                SUM(total_visits) as total_visits,
                SUM(total_spent) as total_revenue,
                AVG(total_spent) as avg_spent,
                SUM(loyalty_points) as total_points
            FROM customers
        """)
        
        stats = self.cursor.fetchone()
        
        print("\nCUSTOMER STATISTICS:")
        print("="*60)
        print(f"Total Customers: {stats[0]}")
        print(f"Total Visits: {stats[1]}")
        print(f"Total Revenue: ₹{stats[2]:.2f}")
        print(f"Average Spent per Customer: ₹{stats[3]:.2f}")
        print(f"Total Loyalty Points: {stats[4]}")
        print("="*60)
        
        # Top customers
        print("\nTOP 10 CUSTOMERS BY SPENDING:")
        print("-"*80)
        print(f"{'Rank':<5} {'Name':<25} {'Email':<30} {'Visits':<8} {'Spent':<12}")
        print("-"*80)
        
        self.cursor.execute("""
            SELECT name, email, total_visits, total_spent
            FROM customers
            ORDER BY total_spent DESC
            LIMIT 10
        """)
        
        for i, (name, email, visits, spent) in enumerate(self.cursor.fetchall(), 1):
            print(f"{i:<5} {name[:24]:<25} {email[:29]:<30} {visits:<8} ₹{spent:<10.2f}")
        
        print("="*80)

    def generate_modifications_report(self):
        """Generate modifications popularity report"""
        print("\n=== MODIFICATIONS REPORT ===")
        
        self.cursor.execute("""
            SELECT 
                bi.mod_category,
                bi.mod_name,
                COUNT(*) as times_sold,
                SUM(bi.quantity) as total_quantity,
                SUM(bi.total_price) as total_revenue,
                AVG(bi.price) as avg_price
            FROM bill_items bi
            GROUP BY bi.mod_category, bi.mod_name
            ORDER BY total_revenue DESC
            LIMIT 20
        """)
        
        mods_data = self.cursor.fetchall()
        
        if not mods_data:
            print("No modification sales data found.")
            return
        
        print("\nTOP SELLING MODIFICATIONS:")
        print("="*90)
        print(f"{'Category':<15} {'Modification':<35} {'Sold':<8} {'Revenue':<15} {'Avg Price':<12}")
        print("-"*90)
        
        total_revenue = 0
        total_sold = 0
        
        for category, name, times_sold, quantity, revenue, avg_price in mods_data:
            print(f"{category:<15} {name[:34]:<35} {times_sold:<8} ₹{revenue:<13.2f} ₹{avg_price:<10.2f}")
            total_revenue += revenue
            total_sold += times_sold
        
        print("-"*90)
        print(f"{'TOTAL':<50} {total_sold:<8} ₹{total_revenue:<13.2f}")
        print("="*90)

    def generate_daily_summary(self):
        """Generate daily summary report"""
        date_str = datetime.now().strftime("%d-%m-%Y")
        
        # Get today's sales
        self.cursor.execute("""
            SELECT 
                COUNT(*) as bills_today,
                SUM(total) as revenue_today,
                AVG(total) as avg_bill_today
            FROM bills
            WHERE DATE(bill_date) = DATE('now')
        """)
        
        sales_today = self.cursor.fetchone()
        
        # Get today's customers
        self.cursor.execute("""
            SELECT COUNT(DISTINCT customer_email) as customers_today
            FROM bills
            WHERE DATE(bill_date) = DATE('now')
        """)
        
        customers_today = self.cursor.fetchone()[0]
        
        # Get today's appointments
        self.cursor.execute("""
            SELECT COUNT(*) as appointments_today
            FROM appointments
            WHERE appointment_date = DATE('now') AND status = 'Scheduled'
        """)
        
        appointments_today = self.cursor.fetchone()[0]
        
        print(f"\n=== DAILY SUMMARY - {date_str} ===")
        print("="*60)
        print(f"Bills Today: {sales_today[0] or 0}")
        print(f"Revenue Today: ₹{sales_today[1] or 0:.2f}")
        print(f"Average Bill: ₹{sales_today[2] or 0:.2f}")
        print(f"Customers Today: {customers_today}")
        print(f"Appointments Today: {appointments_today}")
        print("="*60)

    def generate_monthly_report(self):
        """Generate monthly report"""
        year = datetime.now().year
        month = datetime.now().month
        
        self.cursor.execute("""
            SELECT 
                strftime('%Y-%m', bill_date) as month_year,
                COUNT(*) as total_bills,
                SUM(total) as total_revenue,
                AVG(total) as avg_bill,
                COUNT(DISTINCT customer_email) as unique_customers,
                SUM(discount) as total_discount
            FROM bills
            WHERE strftime('%Y', bill_date) = ?
            GROUP BY strftime('%Y-%m', bill_date)
            ORDER BY month_year DESC
            LIMIT 12
        """, (str(year),))
        
        monthly_data = self.cursor.fetchall()
        
        if not monthly_data:
            print("No monthly data found.")
            return
        
        print(f"\n=== MONTHLY REPORT - {year} ===")
        print("="*120)
        print(f"{'Month':<10} {'Bills':<8} {'Revenue':<15} {'Avg Bill':<12} {'Customers':<10} {'Discount':<12} {'GST':<12} {'Net':<15}")
        print("-"*120)
        
        for month_year, bills, revenue, avg_bill, customers, discount in monthly_data:
            gst = revenue * (GST_RATE / (1 + GST_RATE))  # Approximate GST calculation
            net = revenue - gst
            month_display = datetime.strptime(month_year, "%Y-%m").strftime("%b %Y")
            print(f"{month_display:<10} {bills:<8} ₹{revenue:<13.2f} ₹{avg_bill:<10.2f} {customers:<10} ₹{discount:<10.2f} ₹{gst:<10.2f} ₹{net:<13.2f}")
        
        print("="*120)

    # -------------------------
    # DATA EXPORT
    # -------------------------

    def export_data_menu(self):
        """Menu for exporting data"""
        while True:
            print("\n=== EXPORT DATA ===")
            print("1. Export Customer Data (CSV)")
            print("2. Export Sales Data (CSV)")
            print("3. Export Modifications Data (CSV)")
            print("4. Export All Data (JSON)")
            print("5. Back to Main Menu")
            
            choice = self.get_int_input("Select option: ", 1, 5)
            
            if choice == 1:
                self.export_customer_data()
            elif choice == 2:
                self.export_sales_data()
            elif choice == 3:
                self.export_modifications_data()
            elif choice == 4:
                self.export_all_data()
            elif choice == 5:
                break

    def export_customer_data(self):
        """Export all customer data to a CSV file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"exports/customers_{timestamp}.csv"
        
        # Get all customer data
        self.cursor.execute("""
            SELECT email, name, phone, address, city, state, pincode,
                   total_visits, total_spent, loyalty_points, 
                   first_visit, last_visit
            FROM customers
            ORDER BY last_visit DESC
        """)
        
        customers = self.cursor.fetchall()
        
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Email', 'Name', 'Phone', 'Address', 'City', 
                           'State', 'Pincode', 'Total Visits', 'Total Spent', 
                           'Loyalty Points', 'First Visit', 'Last Visit'])
            
            for customer in customers:
                writer.writerow(customer)
        
        print(f"✓ Customer data exported to {filename}")
        print(f"  Total records: {len(customers)}")

    def export_sales_data(self):
        """Export sales data to a CSV file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"exports/sales_{timestamp}.csv"
        
        # Get all sales data
        self.cursor.execute("""
            SELECT b.bill_id, b.customer_email, c.name, ca.car_model,
                   b.bill_date, b.bill_number, b.subtotal, b.discount,
                   b.gst, b.total, b.payment_method, b.notes
            FROM bills b
            JOIN customers c ON b.customer_email = c.email
            LEFT JOIN cars ca ON b.car_id = ca.car_id
            ORDER BY b.bill_date DESC
        """)
        
        sales = self.cursor.fetchall()
        
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Bill ID', 'Customer Email', 'Customer Name', 'Car Model',
                           'Bill Date', 'Bill Number', 'Subtotal', 'Discount',
                           'GST', 'Total', 'Payment Method', 'Notes'])
            
            for sale in sales:
                writer.writerow(sale)
        
        print(f"✓ Sales data exported to {filename}")
        print(f"  Total records: {len(sales)}")

    def export_modifications_data(self):
        """Export modifications sales data to a CSV file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"exports/modifications_{timestamp}.csv"
        
        # Get modifications sales data
        self.cursor.execute("""
            SELECT b.bill_date, b.bill_id, c.name, ca.car_model,
                   bi.mod_category, bi.mod_name, bi.quantity, bi.price, bi.total_price
            FROM bill_items bi
            JOIN bills b ON bi.bill_id = b.bill_id
            JOIN customers c ON b.customer_email = c.email
            LEFT JOIN cars ca ON b.car_id = ca.car_id
            ORDER BY b.bill_date DESC
        """)
        
        mods = self.cursor.fetchall()
        
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Bill Date', 'Bill ID', 'Customer Name', 'Car Model',
                           'Category', 'Modification', 'Quantity', 'Unit Price', 'Total Price'])
            
            for mod in mods:
                writer.writerow(mod)
        
        print(f"✓ Modifications data exported to {filename}")
        print(f"  Total records: {len(mods)}")

    def export_all_data(self):
        """Export all data to a JSON file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"exports/full_backup_{timestamp}.json"
        
        data = {
            'export_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'customers': [],
            'bills': [],
            'bill_items': [],
            'cars': [],
            'modifications': []
        }
        
        # Export customers
        self.cursor.execute("SELECT * FROM customers")
        columns = [description[0] for description in self.cursor.description]
        for row in self.cursor.fetchall():
            data['customers'].append(dict(zip(columns, row)))
        
        # Export bills
        self.cursor.execute("SELECT * FROM bills")
        columns = [description[0] for description in self.cursor.description]
        for row in self.cursor.fetchall():
            data['bills'].append(dict(zip(columns, row)))
        
        # Export bill items
        self.cursor.execute("SELECT * FROM bill_items")
        columns = [description[0] for description in self.cursor.description]
        for row in self.cursor.fetchall():
            data['bill_items'].append(dict(zip(columns, row)))
        
        # Export cars
        self.cursor.execute("SELECT * FROM cars")
        columns = [description[0] for description in self.cursor.description]
        for row in self.cursor.fetchall():
            data['cars'].append(dict(zip(columns, row)))
        
        # Export modifications
        self.cursor.execute("SELECT * FROM modifications")
        columns = [description[0] for description in self.cursor.description]
        for row in self.cursor.fetchall():
            data['modifications'].append(dict(zip(columns, row)))
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, default=str)
        
        print(f"✓ Full data backup exported to {filename}")
        print(f"  Total records: {len(data['customers'])} customers, {len(data['bills'])} bills")

    # -------------------------
    # CLEANUP
    # -------------------------

    def __del__(self):
        """Destructor to ensure database connection is closed"""
        if hasattr(self, 'conn'):
            self.conn.close()


# =========================
# PROGRAM START
# =========================

if __name__ == "__main__":
    print("\n" + "="*80)
    print(" " * 25 + "🚗 CUSTOM CAR MODIFICATION STUDIO 🚗")
    print("="*80)
    print("Database: car_mod.db")
    print("="*80)
    
    studio = ModificationStudio()
    
    try:
        studio.menu()
    except KeyboardInterrupt:
        print("\n\nProgram interrupted by user. Saving data...")
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        print("Data has been saved to the database.")
    finally:
        if hasattr(studio, 'conn'):
            studio.conn.close()
        print("\nKeppp revvingg !!. Goodbye!")


                         🚗 CUSTOM CAR MODIFICATION STUDIO 🚗
Database: car_mod.db


OperationalError: unable to open database file


1. New Modification Build
2. Exit


Choose option:  2


Keepp revvinggg !!
